In [2]:
import pandas as pd
import numpy as np
import random

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Sample data options
parts = [
    "Main Landing Gear", "Nose Gear", "Hydraulic Actuator", "Brake Assembly",
    "Steering Unit", "Strut", "Tire", "Shock Absorber", "Drag Brace", "Retract Actuator"
]

airlines = ["AirGlobal", "SkyFleet", "AeroLine", "CloudJet", "FlyHigh"]

damage_types = [
    "Normal Wear", "Foreign Object Damage (FOD)", "Customer Mishandling",
    "Corrosion", "Unknown", "Overhaul Required"
]

# Damage-based contract coverage rules
def is_billable(damage):
    billable = ["Foreign Object Damage (FOD)", "Customer Mishandling", "Unknown"]
    return "Yes" if damage in billable else "No"

# Generate synthetic dataset
num_records = 200
data = []

for i in range(num_records):
    part = random.choice(parts)
    airline = random.choice(airlines)
    damage = random.choice(damage_types)
    cost = round(np.random.uniform(1500, 25000), 2)  # repair cost
    teardown_note = f"Teardown revealed: {damage.lower()}."
    billable = is_billable(damage)

    data.append({
        "Repair ID": f"R{i+1:04}",
        "Part": part,
        "Airline": airline,
        "Damage Type": damage,
        "Repair Cost (USD)": cost,
        "Teardown Report": teardown_note,
        "Billable to Customer": billable
    })

# Create DataFrame
df = pd.DataFrame(data)

# Preview
print(df.head())

# Optional: Save to CSV
df.to_csv("repair_events_dataset.csv", index=False)


  Repair ID                Part    Airline                  Damage Type  \
0     R0001           Nose Gear  AirGlobal            Overhaul Required   
1     R0002       Steering Unit   SkyFleet  Foreign Object Damage (FOD)   
2     R0003  Hydraulic Actuator  AirGlobal            Overhaul Required   
3     R0004          Drag Brace  AirGlobal                      Unknown   
4     R0005                Tire  AirGlobal                  Normal Wear   

   Repair Cost (USD)                                  Teardown Report  \
0           10301.69            Teardown revealed: overhaul required.   
1           23841.79  Teardown revealed: foreign object damage (fod).   
2           18701.86            Teardown revealed: overhaul required.   
3           15568.47                      Teardown revealed: unknown.   
4            5166.44                  Teardown revealed: normal wear.   

  Billable to Customer  
0                   No  
1                  Yes  
2                   No  
3         

In [4]:
# Function to apply detailed rule logic
def apply_contract_rules(row):
    damage = row["Damage Type"]
    cost = row["Repair Cost (USD)"]

    if damage == "Normal Wear":
        return "R1", "Covered under standard maintenance plan", 0, cost
    elif damage == "Foreign Object Damage (FOD)":
        return "R2", "Customer liability", cost, 0
    elif damage == "Customer Mishandling":
        return "R3", "Negligence not covered by contract", cost, 0
    elif damage == "Corrosion":
        return "R4", "Covered if routine maintenance documented", 0, cost
    elif damage in ["Unknown", "Overhaul Required"]:
        return "R5", "Investigated case; charged to customer", cost, 0
    else:
        return "R0", "Undefined damage type", cost, 0

# Apply logic to each row
df[["Contract Rule ID", "Justification", "Billable Amount", "Covered Amount"]] = df.apply(
    lambda row: pd.Series(apply_contract_rules(row)),
    axis=1
)

# Preview updated DataFrame
print(df.head())

# Save to new CSV
df.to_csv("repair_events_with_rules.csv", index=False)


  Repair ID                Part    Airline                  Damage Type  \
0     R0001           Nose Gear  AirGlobal            Overhaul Required   
1     R0002       Steering Unit   SkyFleet  Foreign Object Damage (FOD)   
2     R0003  Hydraulic Actuator  AirGlobal            Overhaul Required   
3     R0004          Drag Brace  AirGlobal                      Unknown   
4     R0005                Tire  AirGlobal                  Normal Wear   

   Repair Cost (USD)                                  Teardown Report  \
0           10301.69            Teardown revealed: overhaul required.   
1           23841.79  Teardown revealed: foreign object damage (fod).   
2           18701.86            Teardown revealed: overhaul required.   
3           15568.47                      Teardown revealed: unknown.   
4            5166.44                  Teardown revealed: normal wear.   

  Billable to Customer Contract Rule ID  \
0                   No               R5   
1                  Yes  

In [5]:
# Total billable and covered amounts per airline
airline_summary = df.groupby("Airline")[["Billable Amount", "Covered Amount"]].sum().sort_values("Billable Amount", ascending=False)

# Display
print(airline_summary)

           Billable Amount  Covered Amount
Airline                                   
SkyFleet         489898.09       231693.33
FlyHigh          369081.79       218073.50
AirGlobal        355468.19        89152.80
CloudJet         279160.40       123041.86
AeroLine         269315.28       149944.03


In [7]:
# a total repair cost column
df["Total Repair Cost"] = df["Billable Amount"] + df["Covered Amount"]

# Group by airline and calculate recovery rate
recovery = df.groupby("Airline").agg({
    "Billable Amount": "sum",
    "Total Repair Cost": "sum"
})
recovery["Recovery Rate (%)"] = (recovery["Billable Amount"] / recovery["Total Repair Cost"]) * 100
recovery = recovery.sort_values("Recovery Rate (%)", ascending=False)

print(recovery)


           Billable Amount  Total Repair Cost  Recovery Rate (%)
Airline                                                         
AirGlobal        355468.19          444620.99          79.948585
CloudJet         279160.40          402202.26          69.407964
SkyFleet         489898.09          721591.42          67.891341
AeroLine         269315.28          419259.31          64.235969
FlyHigh          369081.79          587155.29          62.859314


In [8]:
damage_counts = df["Damage Type"].value_counts()

print(damage_counts)

Damage Type
Overhaul Required              39
Normal Wear                    38
Foreign Object Damage (FOD)    33
Unknown                        32
Customer Mishandling           32
Corrosion                      26
Name: count, dtype: int64


In [9]:
billable_by_part = df.groupby("Part")["Billable Amount"].sum().sort_values(ascending=False)

print(billable_by_part)


Part
Shock Absorber        266854.09
Drag Brace            243939.21
Steering Unit         225386.16
Tire                  204265.34
Nose Gear             175452.24
Hydraulic Actuator    159923.14
Strut                 146896.90
Brake Assembly        146414.84
Retract Actuator      100141.54
Main Landing Gear      93650.29
Name: Billable Amount, dtype: float64
